In [ ]:
#Download specific submissions from Reddit defiend by parameters after, before, and title

In [5]:
#import all important libraries

#general libraries
import os 
import pandas as pd


#data format libraries
import json
import csv

#time libraries
import datetime as dt
import time

#API connection
from pmaw import PushshiftAPI
import requests


In [17]:
#define function for downloading submmissions with specific parameters
def getPushshiftData(after, before, title, limit):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(title)+'&after='+str(after)+'&before='+str(before)+'&limit='+str(limit)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [ ]:
#set parameters according to own preferences
title="(black%20lives%20matter)|(george%20floyd)" #search strings in title
before = "1621979999" #May 25th 2021
after = "1590357600"  #May 25th 2020 --> George Floyds death date    
limit = "1000" #returns 1000 items each query
#subData = []
subCount = 0
subStats = {}
            
            
#get submission request with specified parameters
data = getPushshiftData(after, before, title, limit)


In [72]:
#funciton for collecting all necessary information for one submission  
#add try and except statement for preventing code failure in case information is not provided
def collectSubData(subm):
    subData = list()
    title = subm['title']
    url = subm['url'] 
    author = subm['author']
    try:
        author_fullname = subm['author_fullname']
    except KeyError:
        author_fullname = "NaN"
    try:
        author_premium = subm['author_premium']
    except KeyError:
        author_premium = "NaN"
    try:
        domain = subm['domain']
    except KeyError:
        domain = "NaN"
    try:
        selftext = subm['selftext']
    except KeyError:
        selftext = "NaN"
    sub_id = subm['id']
    subreddit_subscribers = subm['subreddit_subscribers']
    score = subm['score']
    upvote_ratio = subm['upvote_ratio']
    subreddit = subm['subreddit']
    subreddit_id = subm['subreddit_id']
    created = datetime.datetime.fromtimestamp(subm['created_utc'])
    retrieved = datetime.datetime.fromtimestamp(subm['retrieved_on'])
    numComms = subm['num_comments']
    num_crossposts = subm['num_crossposts']
    permalink = subm['permalink']
    full_link = subm['full_link']
    thumbnail = subm['thumbnail']
    try:
        pinned = subm['pinned']
    except KeyError:
        pinned = "NaN"
    try:
        stickied = subm['stickied']
    except KeyError:
        stickied = "NaN"
    try:
        is_video = subm['is_video']
    except KeyError:
        is_video = "NaN"
    try:
        media_only = subm['media_only']
    except KeyError:
        media_only = "NaN"
    try:
        over_18 = subm['over_18']
    except KeyError:
        over_18 = "NaN"
    try:
        is_reddit_media_domain = subm['is_reddit_media_domain']
    except KeyError:
        is_reddit_media_domain = "NaN"
    try:
        is_original_content = subm['is_original_content']
    except KeyError:
        is_original_content = "NaN"
    try:
        is_crosspostable = subm['is_crosspostable']
    except KeyError:
        is_crosspostable = "NaN"
    try:
        link_flair_text = subm['link_flair_text']
    except KeyError:
        link_flair_text = "NaN"
    try:
        link_flair_type = subm['link_flair_type']
    except KeyError:
        link_flair_type = "NaN"
    try:
        total_awards_received = subm['total_awards_received']
    except KeyError:
        total_awards_received = "NaN"
    try:
        media = subm['media']
    except KeyError:
        media = "NaN"
    
    
    #append all data to list
    subData.append((sub_id,title,url,author,author_fullname,author_premium,selftext,subreddit,subreddit_id,\
                    subreddit_subscribers,domain,score,upvote_ratio,created,retrieved,numComms,num_crossposts,\
                    permalink,full_link,thumbnail,pinned,stickied,is_video,media_only,over_18,\
                    is_reddit_media_domain,is_original_content,is_crosspostable,link_flair_text,link_flair_type,\
                    total_awards_received,media))
    subStats[sub_id] = subData

In [ ]:
#create while loop that will run until all posts in specific time frame have been collected 

while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    
    #calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    try:
        data = getPushshiftData(after, before, title, limit)
    except Exception as e:
        print("Error")
    
    
print(len(data))

In [ ]:
#turn the dictionary values into a list and show first and last entry
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

In [1]:
#save data to CSV file
def updateSubs_file():
    upload_count = 0
    location = "/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data"
    print("input filename of submission file, please add .csv")
    filename = input()
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=';')
        headers = ["sub_id","title","url","author","author_fullname","author_premium","selftext","subreddit",\
                   "subreddit_id","subreddit_subscribers","domain","score","upvote_ratio","created",\
                   "retrieved","number_of_comments","number_crossposts","permalink","full_Link","thumbnail",\
                   "pinned","stickied","is_video","media_only","over_18","is_reddit_media_domain",\
                   "is_original_content", "is_crosspostable","link_flair_text", "link_flair_type",\
                   "total_awards_received","media"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()


